In [ ]:
!pip install -U awswrangler

In [ ]:
import awswrangler as wr
import boto3
from datetime import datetime, timedelta
import logging
import pandas as pd
from pathlib import Path 
from typing import List

In [ ]:
def generate_crosswalk_report(input_list:List, s3_output_results_path:str, database_name:str, database_workgroup:str) -> None:
    today = datetime.now().strftime("%Y-%m-%d")
    query_strs = []
    folder_names = []
    for item in input_list:
        path = Path(item)
        if path.is_dir():
            logging.info("utilizing directory")
            for sql_file_path in sorted(path.glob("*.sql")):
                with sql_file_path.open('r') as f:
                    query_str = f.read()
                folder_names.append(sql_file_path.name.replace('.sql', ''))
                query_strs.append(query_str)
                
    for folder, query in zip(folder_names, query_strs):
        df = wr.athena.read_sql_query(sql=query, database=database_name, workgroup=database_workgroup)
        wr.s3.to_csv(df, f"{s3_output_results_path}{today}/{folder}.csv", index=False)


In [ ]:
if __name__ == "__main__":
    input_list = ['/root/MARS_ETL_PROCESSES/sql/crosswalk_reports/']
    s3_output_results_path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_data_quality_reports/crosswalk_reports/'
    database_name = 'sandbox_measurement_analyst'
    database_workgroup = 'cmg-oasis-prod-measurement_analyst-wkgrp'
    
    generate_crosswalk = generate_crosswalk_report(input_list, s3_output_results_path, database_name, database_workgroup)
    